# MLP

In [1]:
import sys
sys.path.append("..")
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from data_prepocess import load_training_data, load_test_data, normalize_features
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import time
import joblib

In [5]:
# Get the data
X_train, y_train = load_training_data('../data/train.csv')

# Standardize features instead of just normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Initialize variables
hidden_layer_sizes = [(128,64,32), (64,32,16), (32,16,8)]
alpha_values = [0.001, 0.01, 0.1]
learning_rate_init_values = [0.005, 0.01, 0.05] 
max_iter_values = [500]
results = {}

# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=['hidden_layer_size', 'alpha', 'learning_rate_init', 'max_iter', 'accuracy', 'f1_score'])

# Define 5-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True)

# Perform 5-fold cross validation
for hidden_layer_size in hidden_layer_sizes:
    for alpha in alpha_values:
        for learning_rate_init in learning_rate_init_values:
            for max_iter in max_iter_values:
                start_time = time.time()

                # Define the model with early stopping
                model = MLPClassifier(hidden_layer_sizes=hidden_layer_size, activation='relu', solver='adam',
                                      max_iter=max_iter, alpha=alpha, learning_rate_init=learning_rate_init,
                                      early_stopping=True, n_iter_no_change=10)

                # Perform cross-validation manually to get the number of iterations
                cv_results = []
                f1_results = []
                for train_index, test_index in kfold.split(X_train):
                    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
                    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
                    model.fit(X_train_fold, y_train_fold)
                    score = model.score(X_test_fold, y_test_fold)
                    cv_results.append(score)
                    y_pred = model.predict(X_test_fold)
                    f1 = f1_score(y_test_fold, y_pred, average='weighted')
                    f1_results.append(f1)

                # Save the model
                model_directory = "model_validation"
                model_filename = f"{model_directory}/model_{hidden_layer_size}_{alpha}_{learning_rate_init}_{max_iter}.joblib"
                joblib.dump(model, model_filename)

                end_time = time.time()
                elapsed_time = end_time - start_time
                config = (hidden_layer_size, alpha, learning_rate_init, max_iter)
                print("Config: {}\nCross-validation mean accuracy: {:.2f}%\nElapsed time: {:.2f} seconds\nNumber of iterations: {}\n".format(
                    config, np.mean(cv_results)*100, elapsed_time, model.n_iter_))

                new_row = pd.DataFrame({
                    'hidden_layer_size': [hidden_layer_size],
                    'alpha': [alpha],
                    'learning_rate_init': [learning_rate_init],
                    'max_iter': [max_iter],
                    'accuracy': [np.mean(cv_results)],
                    'f1_score': [np.mean(f1_results)]
                })

                results_df = pd.concat([results_df, new_row], ignore_index=True)

# Save the results to a CSV file
results_df.to_csv('model_results.csv', index=False)

Config: ((128, 64, 32), 0.001, 0.005, 500)
Cross-validation mean accuracy: 56.30%
Elapsed time: 0.77 seconds
Number of iterations: 12



C:\Users\10150\AppData\Local\Temp\ipykernel_8456\2702057061.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


Config: ((128, 64, 32), 0.001, 0.01, 500)
Cross-validation mean accuracy: 55.77%
Elapsed time: 0.78 seconds
Number of iterations: 25

Config: ((128, 64, 32), 0.001, 0.05, 500)
Cross-validation mean accuracy: 58.09%
Elapsed time: 0.76 seconds
Number of iterations: 15

Config: ((128, 64, 32), 0.01, 0.005, 500)
Cross-validation mean accuracy: 57.64%
Elapsed time: 0.92 seconds
Number of iterations: 31

Config: ((128, 64, 32), 0.01, 0.01, 500)
Cross-validation mean accuracy: 60.32%
Elapsed time: 0.71 seconds
Number of iterations: 20

Config: ((128, 64, 32), 0.01, 0.05, 500)
Cross-validation mean accuracy: 57.55%
Elapsed time: 0.60 seconds
Number of iterations: 13

Config: ((128, 64, 32), 0.1, 0.005, 500)
Cross-validation mean accuracy: 57.10%
Elapsed time: 0.57 seconds
Number of iterations: 17

Config: ((128, 64, 32), 0.1, 0.01, 500)
Cross-validation mean accuracy: 57.82%
Elapsed time: 0.63 seconds
Number of iterations: 17

Config: ((128, 64, 32), 0.1, 0.05, 500)
Cross-validation mean accur

In [3]:
# Get the data
X_train, y_train = load_training_data('../data/train.csv')

# Convert y_train to binary
# y_train = np.where(y_train > 6, 1, 0)

# Standardize features instead of just normalizing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Initialize variables
hidden_layer_sizes = [(128,64,32), (64,32,16), (32,16,8)]
alpha_values = [0.001, 0.01, 0.1]
learning_rate_init_values = [0.005, 0.01, 0.05] 
max_iter_values = [500]
results = {}

# Define 5-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True)

# Perform 5-fold cross validation
for hidden_layer_size in hidden_layer_sizes:
    for alpha in alpha_values:
        for learning_rate_init in learning_rate_init_values:
            for max_iter in max_iter_values:
                start_time = time.time()

                # Define the model with early stopping
                model = MLPClassifier(hidden_layer_sizes=hidden_layer_size, activation='relu', solver='adam',
                                      max_iter=max_iter, alpha=alpha, learning_rate_init=learning_rate_init,
                                      early_stopping=True, n_iter_no_change=10)

                # Perform cross-validation manually to get the number of iterations
                cv_results = []
                for train_index, test_index in kfold.split(X_train):
                    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
                    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
                    model.fit(X_train_fold, y_train_fold)
                    score = model.score(X_test_fold, y_test_fold)
                    cv_results.append(score)

                # Save the model
                model_directory = "model_validation"
                model_filename = f"{model_directory}/model_{hidden_layer_size}_{alpha}_{learning_rate_init}_{max_iter}.joblib"
                joblib.dump(model, model_filename)

                end_time = time.time()
                elapsed_time = end_time - start_time
                config = (hidden_layer_size, alpha, learning_rate_init, max_iter)
                print("Config: {}\nCross-validation mean accuracy: {:.2f}%\nElapsed time: {:.2f} seconds\nNumber of iterations: {}\n".format(
                    config, np.mean(cv_results)*100, elapsed_time, model.n_iter_))

                results[config] = np.mean(cv_results)

print("Results:", results)

Config: ((128, 64, 32), 0.001, 0.005, 500)
Cross-validation mean accuracy: 60.41%
Elapsed time: 1.00 seconds
Number of iterations: 20

Config: ((128, 64, 32), 0.001, 0.01, 500)
Cross-validation mean accuracy: 57.55%
Elapsed time: 0.73 seconds
Number of iterations: 39

Config: ((128, 64, 32), 0.001, 0.05, 500)
Cross-validation mean accuracy: 56.21%
Elapsed time: 0.63 seconds
Number of iterations: 21

Config: ((128, 64, 32), 0.01, 0.005, 500)
Cross-validation mean accuracy: 58.72%
Elapsed time: 0.87 seconds
Number of iterations: 26

Config: ((128, 64, 32), 0.01, 0.01, 500)
Cross-validation mean accuracy: 55.49%
Elapsed time: 0.93 seconds
Number of iterations: 14

Config: ((128, 64, 32), 0.01, 0.05, 500)
Cross-validation mean accuracy: 56.92%
Elapsed time: 0.71 seconds
Number of iterations: 25

Config: ((128, 64, 32), 0.1, 0.005, 500)
Cross-validation mean accuracy: 58.26%
Elapsed time: 0.91 seconds
Number of iterations: 22

Config: ((128, 64, 32), 0.1, 0.01, 500)
Cross-validation mean ac

## Model's paramemters with the best accuracy

In [ ]:
# Find the configuration with the highest accuracy
best_config = max(results, key=results.get)
best_accuracy = results[best_config]

# Print the best configuration and its accuracy
print("Best configuration: ", best_config)
print("Best cross-validation mean accuracy: {:.2f}%".format(best_accuracy*100))

Best configuration:  ((128, 64, 32), 0.1, 0.005, 1500)
Best cross-validation mean accuracy: 60.77%


## Acuracy on the test set

In [ ]:
from sklearn.metrics import f1_score

# Load the test data
X_test, y_test = load_test_data('../data/test.csv')

# Normalize the test data
X_test = normalize_features(X_test)

# Get the best configuration
best_config = max(results, key=results.get)

# Parse the best configuration
hidden_layer_size, alpha, learning_rate_init, max_iter = best_config

# Load the trained model with the best configuration
model_directory = "model_validation"
model_filename = f"{model_directory}/model_{hidden_layer_size}_{alpha}_{learning_rate_init}_{max_iter}.joblib"
model = joblib.load(model_filename)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test data: {:.2f}%".format(accuracy * 100))

# Calculate and print the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score on test data: {:.2f}".format(f1))

Accuracy on test data: 58.33%
F1 score on test data: 0.54


c:\Users\10150\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
